In [14]:
# Travel time cell
from datetime import datetime, timedelta
from pytz import timezone as ptz

DATE_FORMAT = "%Y-%m-%dT%H:%M:%-SZ"
oneDay = timedelta(1)

departureTimeOptions = {
    'now': lambda tz: datetime.now(tz).replace(minute=59, second=59).isoformat(),
    'morning': lambda tz: (datetime.now(tz).replace(hour=9, minute=0, second=0) + oneDay).isoformat(),
    'noon': lambda tz: (datetime.now(tz).replace(hour=12, minute=0, second=0) + oneDay).isoformat(),
    'afternoon': lambda tz: (datetime.now(tz).replace(hour=5, minute=0, second=0) + oneDay).isoformat(),
    'evening': lambda tz: (datetime.now(tz).replace(hour=7, minute=0, second=0) + oneDay).isoformat(),
}

print("now time: " + departureTimeOptions['now'](ptz('EST')))
print("morning time: " + departureTimeOptions['morning'](ptz('EST')))
print(datetime.now().timestamp())

now time: 2023-09-29T17:59:59.569783-05:00
morning time: 2023-09-30T09:00:00.570264-05:00
1696028067.570329


In [20]:
import math
import requests
from const import GOOGLE_MAPS_API_KEY
import json


COMPUTE_ROUTE_URL = 'https://routes.googleapis.com/directions/v2:computeRoutes'
MY_HOUSE = '3025 The Credit Woodlands, Mississauga, ON L5C 2V3'
SQUARE_ONE_ADDRESS = '100 City Centre Dr, Mississauga, ON L5B 2C9'
# travelTimes = {
#     now: lambda : datetime.datetime.now()
#     morning: lambda : datetime.datetime
# }

getDurationSec = lambda route: int(route["duration"][0:-1])

def findFastestRoute(response) -> timedelta:
    routeDurations = json.loads(response.text)["routes"]
    fastestCommute = math.inf

    for r in routeDurations:
        fastestCommute = min(getDurationSec(r), fastestCommute)

    return timedelta(seconds=fastestCommute)

def computeDrivingRoute(originAddress: str, destinationAddress: str, departureOption='now'):

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'DRIVE',
        'routingPreference': 'TRAFFIC_AWARE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
        'computeAlternativeRoutes': False,
        'routeModifiers': {
            'avoidTolls': True,
            'avoidHighways': False,
            'avoidFerries': True,
        },
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return response.text

def computeFastestDriveCommute(originAddress: str, destinationAddress: str, departureOption='now') -> timedelta:

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.duration',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'DRIVE',
        'routingPreference': 'TRAFFIC_AWARE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
        'computeAlternativeRoutes': True,
        'routeModifiers': {
            'avoidTolls': True,
            'avoidHighways': False,
            'avoidFerries': True,
        },
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return findFastestRoute(response)



# print(computeDrivingRoute(MY_HOUSE, SQUARE_ONE_ADDRESS))
print("Fastest Driving Commute: ")
print(computeFastestDriveCommute(MY_HOUSE, SQUARE_ONE_ADDRESS))

Fastest Driving Commute: 
0:15:45


In [16]:


def computeTransitRoute(originAddress: str, destinationAddress: str, departureOption='now'):
    headers = {
        'content-type': 'application/json; application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.legs.steps.transitDetails',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'TRANSIT',
        'computeAlternativeRoutes': True,
        'transitPreferences': {
            'routingPreference': 'FEWER_TRANSFERS',
        },
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return response.text

def computeFastestTransitCommute(originAddress: str, destinationAddress: str, departureOption='now'):
    headers = {
        'content-type': 'application/json; application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.duration',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'TRANSIT',
        'computeAlternativeRoutes': True,
        'transitPreferences': {
            'routingPreference': 'FEWER_TRANSFERS',
        },
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return findFastestRoute(response)

# print(computeTransitRoute(MY_HOUSE, SQUARE_ONE_ADDRESS))
print("Fastest transit commute: ")
print(computeFastestTransitCommute(MY_HOUSE, SQUARE_ONE_ADDRESS))


Fastest transit commute: 
0:32:26


In [17]:

def computeBicyleRoutes(originAddress: str, destinationAddress: str, departureOption='now'):

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'BICYCLE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return response.text


def computeFastestBicyleCommute(originAddress: str, destinationAddress: str, departureOption='now'):

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.duration',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'BICYCLE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return findFastestRoute(response)


# print(computeBicyleRoutes(MY_HOUSE, SQUARE_ONE_ADDRESS))
print("Fastest Bicycle Commute:")
print(computeFastestBicyleCommute(MY_HOUSE, SQUARE_ONE_ADDRESS))

Fastest Bicycle Commute:
0:27:16
